In [6]:
import sys
sys.path.insert(1, '../..')

import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv("../../data/processed/phemernr2-tf_dataset.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,topic,tvt2,tvt2_1,tvt2_2,tvt2_3
0,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,True,germanwings-crash-all-rnr-threads,training,training,training,training
1,552798891994009601,Reports that two of the dead in the #CharlieHe...,True,charliehebdo-all-rnr-threads,training,training,training,training
2,580352540316946432,'No survivors' in #Germanwings crash says Fren...,False,germanwings-crash-all-rnr-threads,training,training,training,testting
3,524983403775799297,Tragedy mounts as soldier shot this AM dies of...,True,ottawashooting-all-rnr-threads,training,training,validation,training
4,544511915158810624,Watch the moment gunfire and explosions were h...,True,sydneysiege-all-rnr-threads,training,training,training,testting


In [8]:
data['label']

0        True
1        True
2       False
3        True
4        True
        ...  
1700     True
1701     True
1702     True
1703    False
1704     True
Name: label, Length: 1705, dtype: bool

In [9]:
label_rnr = []
for i, d in data.iterrows():
    if d['label'] == True:
        label_rnr.append("true")
    else:
        label_rnr.append("false")
        
data['label_str'] = pd.Series(label_rnr)
data.head()

,tweet_id,tweet_text,label,topic,tvt2,tvt2_1,tvt2_2,tvt2_3,label_str
0,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,True,germanwings-crash-all-rnr-threads,training,training,training,training,true
1,552798891994009601,Reports that two of the dead in the #CharlieHe...,True,charliehebdo-all-rnr-threads,training,training,training,training,true
2,580352540316946432,'No survivors' in #Germanwings crash says Fren...,False,germanwings-crash-all-rnr-threads,training,training,training,testting,false
3,524983403775799297,Tragedy mounts as soldier shot this AM dies of...,True,ottawashooting-all-rnr-threads,training,training,validation,training,true
4,544511915158810624,Watch the moment gunfire and explosions were h...,True,sydneysiege-all-rnr-threads,training,training,training,testting,true


In [10]:
# labels_str = ['rumour', 'non-rumour']
label_type = "label_str"
labels_str = data[label_type].unique().tolist()
labels_str

['true', 'false']

In [11]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d[label_type])
    labels.append(lab)
labels[:10]

[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]

In [12]:
import string
import nltk
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(reduce_len=True)


def text2unigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    unigrams = texts
    
    return unigrams


def text2bigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    bigrams = nltk.bigrams(texts)
    bigrams = map(' '.join, bigrams)
    bigrams = [bgr for bgr in bigrams]
    
    return bigrams


def text2trigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    trigrams = nltk.trigrams(texts)
    trigrams = map(' '.join, trigrams)
    trigrams = [bgr for bgr in trigrams]
    
    return trigrams


def bigrams_vectors_generation(texts):
    bigram_vectors = []
    for text in texts:
        bigrams = text2bigrams(text)

        init_vec = [0.0 for _ in range(len(bigram_vector_base) + 1)]
        for bgr in bigrams:
            if bgr in bigram_vector_base:
                idx = bigram_vector_base.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        bigram_vectors.append(init_vec)
    
    return bigram_vectors


def custom_vectors_generation(texts, vector_terms):
    vectors = []
    for text in texts:
        bigrams = text2bigrams(text)
        trigrams = text2trigrams(text)

        init_vec = [0.0 for _ in range(len(vector_terms) + 1)]
        for bgr in bigrams:
            if bgr in vector_terms:
                idx = vector_terms.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        for tgr in trigrams:
            if tgr in vector_terms:
                idx = vector_terms.index(tgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        vectors.append(init_vec)
    
    return vectors

In [13]:
texts = data['tweet_text'].tolist()
# vectors = bigrams_vectors_generation(texts)

In [14]:
import nltk
from nltk.collocations import *

top_n = 2000
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder2 = BigramCollocationFinder.from_words([])
finder3 = TrigramCollocationFinder.from_words([])

# generating bigram and trigram
for text in texts:
    unigrams = text2unigrams(text)
    bigrams = text2bigrams(text)
    trigrams = text2trigrams(text)
    
    for ngrm in unigrams:
        if ngrm not in finder2.word_fd:
            finder2.word_fd[ngrm] = 0
        finder2.word_fd[ngrm] += 1
        finder2.N += 1
        
        if ngrm not in finder3.word_fd:
            finder3.word_fd[ngrm] = 0
        finder3.word_fd[ngrm] += 1
        finder3.N += 1
    
    for ngrm in bigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder2.ngram_fd:
            finder2.ngram_fd[term] = 0
            
        finder2.ngram_fd[term] += 1

    for ngrm in trigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder3.ngram_fd:
            finder3.ngram_fd[term] = 0
            
        finder3.ngram_fd[term] += 1
        
# only bigrams that appear 3+ times
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(3)

combined = []
for res in finder2.score_ngrams(bigram_measures.pmi):
    combined.append(res)
for res in finder3.score_ngrams(trigram_measures.pmi):
    combined.append(res)
combined = sorted(combined, key=lambda x: x[1], reverse=True)

In [15]:
print(len(finder2.ngram_fd), len(finder3.ngram_fd))
count = 0
for k, v in combined:
    print(f"{k} - {v}")
    count += 1
    if count >= 10:
        break

1548 884
('http://t.co/j1ljDbazOw', 'UPDATES', 'http://t.co/PD7pVfU9pQ') - 25.526534015016388
('New', 'York', 'Times') - 24.78956842085018
("Gurlitt's", 'problematic', 'bequest') - 24.526534015016388
('Photo', 'Thomas', 'Samson') - 24.14802239176266
('Thomas', 'Samson', '#AFP') - 24.14802239176266
('air', 'traffic', 'control') - 24.041107187846144
('posts', 'picture', 'claiming') - 24.018387111346062
('Media', 'account', 'posts') - 23.84846210990375
('AC', 'Milan', 'midfielder') - 23.433424610624904
('account', 'posts', 'picture') - 23.018387111346062


In [16]:
term_vector_base = [" ".join(c[0]) for c in combined[:top_n]]
vectors = custom_vectors_generation(texts, term_vector_base)

In [17]:
len(vectors[102])

2001

In [18]:
min_score = 100
max_score = 0
n_2gram = 0
n_3gram = 0
for k, v in combined[:top_n]:
    min_score = min(min_score, v)
    max_score = max(max_score, v)
    
    if len(k) == 2:
        n_2gram += 1
    
    if len(k) == 3:
        n_3gram += 1
        
print(min_score)
print(max_score)
print(n_2gram)
print(n_3gram)

3.8781386526441306
25.526534015016388
1116
884


In [19]:
train_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

train_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

In [20]:
from sklearn.svm import LinearSVC

from library.classification import SKLearnClassification
from library.evaluation import ConfusionMatrix

svm = LinearSVC()

model = SKLearnClassification(svm, "Support Vector Machine")
print(f"\n--- START ---")
model.train(train_vectors, train_labels, "Phemernr2")

print("Validation Set")
preds = model.predict(val_vectors)

conf_mat = ConfusionMatrix(
    labels=val_labels,
    predictions=preds,
    binary=True
)
conf_mat.evaluate()

print("--- END ---\n")


--- START ---
---> execution time : 0.01 seconds
Validation Set
Binary Class Evaluation

True Positive : 94
False Positive : 24
False Negative : 49
True Negative : 204

Class positive Evaluation
- Precision : 79.661 %
- Recall : 65.734 %
- F1 : 0.72031

Class negative Evaluation
- Precision : 80.632 %
- Recall : 89.474 %
- F1 : 0.84823

Combined Evaluation
- Accuracy : 80.323 %
- Precision : 80.147 %
- Recall : 77.604 %
- F1 : 0.78855
- Average Confidence : 100.0 %
Model, Combined,,,,positive,,,negative,,,
Anonymous, 80.323, 80.147, 77.604, 0.78855, 79.661, 65.734, 0.72031, 80.632, 89.474, 0.84823, 
--- END ---



In [21]:
from scipy.sparse import csr_matrix

coefs = model.model.coef_
if type(coefs) == csr_matrix:
    coefs.toarray().tolist()[0]
else:
    coefs.tolist()
coefs_and_features = list(zip(coefs[0], term_vector_base))

# Most predictive overall
# coefs_and_features = sorted(coefs_and_features, key=lambda x: x[0], reverse=True)
coefs_and_features = sorted(coefs_and_features, key=lambda x: abs(x[0]), reverse=True)
print(f"Total tokens : {len(coefs_and_features)}")

Total tokens : 2000


In [22]:
coefs.shape

(1, 2001)

In [23]:
coefs_and_features

[(1.0626741929281611, 'Massey Hall'),
 (1.0625722576743906, 'in the window'),
 (1.058995986929037, 'Ottawa Police'),
 (1.0160816492639464, 'by @LucilleClerc'),
 (1.0079452736106167, 'CEO says'),
 (1.0047098655115598, 'in Toronto'),
 (0.9849270619103448, '#Prince #Toronto'),
 (0.9829508024967689, 'shooting incident'),
 (-0.981074255776387, '#cbcOTT #OTTnews'),
 (0.9772182729143838, 'Banksy #JeSuisCharlie'),
 (0.9659102383891542, 'was Muslim'),
 (0.9587838759519082, 'take on'),
 (0.9510356757439522, 'person killed'),
 (0.9495631516634369, 'distress call'),
 (0.9272325545539863, '#Germanwings crash'),
 (0.9103565663866631, 'terrorist attack'),
 (0.9075282306430613, 'Police say'),
 (0.892180788339155, 'by Banksy'),
 (-0.8896184990359556, 'to detain'),
 (-0.882543039081298, 'the #CharlieHebdo attack'),
 (-0.8723457603029122, 'in Dammartin-en-Goele'),
 (0.85036763753387, '2 gunmen'),
 (0.848777393203878, '10 times'),
 (-0.8422521475261037, 'so far'),
 (0.8352180224652417, 'held in a'),
 (0.8

In [24]:
coefs_and_features[:10]

[(1.0626741929281611, 'Massey Hall'),
 (1.0625722576743906, 'in the window'),
 (1.058995986929037, 'Ottawa Police'),
 (1.0160816492639464, 'by @LucilleClerc'),
 (1.0079452736106167, 'CEO says'),
 (1.0047098655115598, 'in Toronto'),
 (0.9849270619103448, '#Prince #Toronto'),
 (0.9829508024967689, 'shooting incident'),
 (-0.981074255776387, '#cbcOTT #OTTnews'),
 (0.9772182729143838, 'Banksy #JeSuisCharlie')]

In [25]:
coefs_and_features[-10:]

[(0.0, 'in Germany'),
 (0.0, 'has taken'),
 (0.0, 'memorial has'),
 (0.0, 'to newspaper'),
 (0.0, 'off the'),
 (0.0, 'is holding'),
 (0.0, 'Ottawa identified'),
 (0.0, 'The Canadian'),
 (0.0, 'surprise Massey'),
 (0.0, 'after soldier')]

In [28]:
n_best = 750
best_tokens = []

# for cf in coefs_and_features[-n_best:]:
#     best_tokens.append(cf[1])

for cf in coefs_and_features[:n_best]:
    best_tokens.append(cf[1])
    
print(best_tokens[:10])
    
with open("../../data/processed/phemernr2-tf_best_terms.txt", "w") as f:
    for token in best_tokens:
        f.write(token + "\n")

['Massey Hall', 'in the window', 'Ottawa Police', 'by @LucilleClerc', 'CEO says', 'in Toronto', '#Prince #Toronto', 'shooting incident', '#cbcOTT #OTTnews', 'Banksy #JeSuisCharlie']
